In [58]:
from tensorflow import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow.keras.layers as layers
from sklearn.model_selection import KFold
import scipy as sc

import training_utils

In [41]:
img_size = (240, 320)
img_channels = 3
batch_size = 128
epochs = 100
display_epochs = (0, 100)


In [5]:
model_name = 'VGG16_(256).tf'

In [6]:
train_size = 1400
validation_size = 500
test_size = 100

# Load the DataSet

In [7]:
dataset_path = '/Users/olove/Library/CloudStorage/OneDrive-Personal/AI datasets/CrowdCounter'

In [8]:
labels_df = pd.read_csv(dataset_path + '/labels.csv')
labels_df['image_name'] = labels_df['id'].map('seq_{:06d}.jpg'.format)
labels_df.drop("id", axis=1, inplace=True)
display(labels_df)

,count,image_name
0,35,seq_000001.jpg
1,41,seq_000002.jpg
2,41,seq_000003.jpg
3,44,seq_000004.jpg
4,41,seq_000005.jpg
...,...,...
1995,27,seq_001996.jpg
1996,27,seq_001997.jpg
1997,25,seq_001998.jpg
1998,26,seq_001999.jpg


In [9]:
labels_df = labels_df.sample(frac=1).reset_index(drop=True)
display(labels_df)

,count,image_name
0,31,seq_000963.jpg
1,37,seq_000373.jpg
2,33,seq_001265.jpg
3,43,seq_000985.jpg
4,35,seq_000131.jpg
...,...,...
1995,36,seq_001110.jpg
1996,24,seq_001416.jpg
1997,34,seq_001777.jpg
1998,30,seq_001445.jpg


In [10]:
if (train_size + validation_size + test_size) != len(labels_df):
    print('Dataset size is different from specified class sizes')
    exit(1)

training_df = labels_df[:train_size]
validation_df = labels_df[train_size:train_size + validation_size].reset_index(drop=True)
test_df = labels_df[train_size + validation_size:].reset_index(drop=True)

# Define the model

In [11]:
inputs = keras.Input(shape=img_size + (img_channels,))

## Custom Simple Covnet

Downsizing using strides instead of MaxPolling in order to conserve location data

In [12]:
def basic_covnet_block(input):
    x = layers.Conv2D(filters=32, kernel_size=3, strides=2, activation="relu")(input)
    x = layers.Conv2D(filters=64, kernel_size=3, strides=2, activation="relu")(x)
    x = layers.Conv2D(filters=128, kernel_size=3, strides=2, activation="relu")(x)
    return x

## Custom Depthwise Seperable Convolution

In [13]:
def custom_depthwise_conv_block(input, sizes, repeat_per_size, stride=1, pooling=False, dropout=False, residual=False):
    x = input

    previous_block_activation = x

    for size in sizes:
        for i in range(repeat_per_size - 1):
            x = layers.BatchNormalization()(x)
            x = layers.Activation("relu")(x)
            x = layers.SeparableConv2D(size, 3, padding='same', use_bias=False)(x)
        x = layers.BatchNormalization()(x)
        x = layers.Activation("relu")(x)
        if dropout:
            x = layers.Dropout(0.5)(x)
        x = layers.SeparableConv2D(size, 3, strides=1 if pooling else stride, padding='same', use_bias=False)(x)
        if pooling:
            x = layers.MaxPooling2D(3, strides=stride, padding='same')(x)
        if residual:
            r = layers.SeparableConv2D(size, 1, strides=stride, padding="same")(
                previous_block_activation
            )
            x = layers.add([x, r])
            previous_block_activation = x

    return x


In [79]:
def base_vgg_16_layers(input):
    covnet = keras.applications.vgg16.VGG16(
        include_top=False,
        weights='imagenet',
        input_shape=img_size + (img_channels,))(covnet)
    covnet.trainable = False
    return keras.Model(inputs=input, outputs=covnet)


def output_vgg_16_layers(covnet):
    output = basic_dense_block(covnet, [])
    return keras.Model(inputs=covnet, outputs=output)


def vgg_16_model():
    input = keras.applications.vgg16.preprocess_input(inputs)
    covnet = base_vgg_16_layers()(input)
    return output_vgg_16_layers(covnet, input)


def activate_fine_tuning_on_vgg_16(ft_model):
    # Flag to indicate whether the layers should be trainable
    set_trainable = False

    # Assuming 'vgg16' is the name of the nested VGG16 model
    vgg16 = ft_model.get_layer('vgg16')
    vgg16.trainable = True

    for layer in vgg16.layers:
        # Start fine-tuning from 'block5_conv1'
        if layer.name == 'block5_conv1':
            set_trainable = True

        # Set the trainable flag for the layers
        if set_trainable:
            print(f'Unfreezing layer {layer.name}')
            layer.trainable = True
        else:
            print(f'Freezing layer {layer.name}')
            layer.trainable = False


## Output Layer

In [15]:
def basic_dense_block(covnet_param, sizes, global_pooling=False):
    if global_pooling:
        x = layers.GlobalAveragePooling2D()(covnet_param)
    else:
        x = layers.Flatten()(covnet_param)
    x = layers.Dropout(0.5)(x)
    for size in sizes:
        x = layers.Dense(size, activation="relu")(x)
        x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(1)(x)
    return outputs

In [16]:
def Xception_model():
    covnet = layers.Conv2D(filters=32, kernel_size=5, use_bias=False)(inputs)
    covnet = custom_depthwise_conv_block(covnet, [32, 64], 2, stride=2, pooling=True, residual=True)
    covnet = custom_depthwise_conv_block(covnet, [128], 2, residual=True)
    outputs = basic_dense_block(covnet, [], global_pooling=True)
    return keras.Model(inputs=inputs, outputs=outputs)

In [17]:
def vgg_16_k_fold_validation(model_filename):
    callbacks_list = [
        #    keras.callbacks.EarlyStopping(
        #        monitor="val_loss", patience=4
        #    ),
        keras.callbacks.ModelCheckpoint(
            filepath=model_filename,
            monitor="val_loss",
            save_best_only=True
        ),
        #    keras.callbacks.TensorBoard()
    ]

    i_kf = KFold(n_splits=5, shuffle=True, random_state=2)

    history_store = []

    for i_result in i_kf.split(labels_df):
        i_train = labels_df.iloc[i_result[0]]
        i_test = labels_df.iloc[i_result[1]]

        print(i_result[0])
        print(i_result[1])

        display(labels_df)
        display(i_train)
        print(f'Train size: {len(i_train)}')
        display(i_test)
        print(f'Test size: {len(i_test)}')

        (train_generator, validation_generator) = training_utils.load_generators(i_train, i_test, dataset_path,
                                                                                 batch_size, img_size)

        conv_base = base_vgg_16_layers()

        feature_train = conv_base.predict(train_generator, verbose=1)
        feature_val = conv_base.predict(validation_generator, verbose=1)

        feature_train = np.concatenate(feature_train)
        feature_val = np.concatenate(feature_val)

        display(feature_train)

        i_model = output_vgg_16_layers(feature_train.shape[1:])

        i_model.compile(loss="mse", optimizer="adam", metrics=["mae"])

        i_history = [i_model.fit(feature_train,
                                 epochs=epochs,
                                 callbacks=callbacks_list,
                                 validation_data=feature_val,
                                 ), ]

        activate_fine_tuning_on_vgg_16(conv_base)
        conv_base.add(i_model)
        conv_base.compile(loss="mse", optimizer="adam", metrics=["mae"])

        i_history.append(conv_base.fit(train_generator,
                                       epochs=epochs,
                                       callbacks=callbacks_list,
                                       validation_data=validation_generator,
                                       ))

        history_store.append(i_history)

    return history_store


In [17]:
model = vgg_16_model()
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 240, 320, 3)]     0         
                                                                 
 vgg16 (Functional)          (None, 7, 10, 512)        14714688  
                                                                 
 flatten_1 (Flatten)         (None, 35840)             0         
                                                                 
 dropout_2 (Dropout)         (None, 35840)             0         
                                                                 
 dense_2 (Dense)             (None, 256)               9175296   
                                                                 
 dropout_3 (Dropout)         (None, 256)               0         
                                                                 
 dense_3 (Dense)             (None, 1)                 257 

# Train and compare different models

In [ ]:
model.compile(loss="mse", optimizer="adam", metrics=["mae"])
# TODO: Try mae vs accuracy. mae should be better since we are adjusting it to get closer to the actual value

In [ ]:
callbacks_list = [
    #    keras.callbacks.EarlyStopping(
    #        monitor="val_loss", patience=4
    #    ),
    keras.callbacks.ModelCheckpoint(
        filepath=model_name,
        monitor="val_loss",
        save_best_only=True
    ),
    #    keras.callbacks.TensorBoard()
]

In [ ]:
kf = KFold(n_splits=5, shuffle=True, random_state=2)

result = next(kf.split(labels_df), None)
display(labels_df.iloc[result[0]])

test = labels_df.iloc[result[1]]
display(test)
print(result[0])


In [ ]:
all(labels_df.iloc[result[0]]['image_name'].apply(lambda x: isinstance(x, str)))

In [ ]:
def k_fold_validation(i_model):
    Wsave = i_model.get_weights()

    i_kf = KFold(n_splits=5, shuffle=True, random_state=2)

    history_store = []

    for i_result in i_kf.split(labels_df):
        train = labels_df.iloc[i_result[0]]
        test = labels_df.iloc[i_result[1]]

        print(i_result[0])
        print(i_result[1])

        display(labels_df)
        display(train)
        print(f'Train size: {len(train)}')
        display(test)
        print(f'Test size: {len(test)}')

        (train_generator, validation_generator) = training_utils.load_generators(train, test, dataset_path, batch_size,
                                                                                 img_size)

        i_model.set_weights(Wsave)
        i_history = i_model.fit(train_generator,
                                epochs=epochs,
                                callbacks=callbacks_list,
                                validation_data=validation_generator,
                                )

        history_store.append(i_history)

    return history_store


In [ ]:
#history = k_fold_validation(model)

In [ ]:
history = vgg_16_k_fold_validation(model_name)

In [18]:


callbacks_list = [
    #    keras.callbacks.EarlyStopping(
    #        monitor="val_loss", patience=4
    #    ),
    keras.callbacks.ModelCheckpoint(
        filepath=model_name,
        monitor="val_loss",
        save_best_only=True
    ),
    #    keras.callbacks.TensorBoard()
]

i_kf = KFold(n_splits=5, shuffle=True, random_state=2)

history_store = []

In [19]:
i_result = next(i_kf.split(labels_df), None)
i_train = labels_df.iloc[i_result[0]]
i_test = labels_df.iloc[i_result[1]]

print(i_result[0])
print(i_result[1])

display(labels_df)
display(i_train)
print(f'Train size: {len(i_train)}')
display(i_test)
print(f'Test size: {len(i_test)}')

[   0    1    5 ... 1994 1997 1999]
[   2    3    4    7   37   45   52   61   65   66   68   69   76   80
   82   83   84   85   93  101  112  114  117  119  123  137  142  143
  145  154  155  157  161  173  176  180  181  197  199  215  226  230
  231  232  236  239  240  241  246  248  263  268  270  271  275  278
  280  284  288  294  305  316  323  332  344  352  354  355  356  361
  365  368  370  372  376  385  391  401  411  418  426  445  448  452
  455  456  458  459  465  468  470  472  479  481  484  488  492  494
  501  502  503  505  515  524  526  529  538  540  547  556  557  558
  560  570  577  580  581  582  586  598  600  606  610  611  615  619
  625  627  633  642  643  652  662  675  677  681  694  695  711  728
  735  738  748  768  770  776  786  802  821  823  825  826  835  837
  840  844  849  851  852  871  874  880  882  888  891  897  905  909
  916  923  928  945  951  952  961  969  978  980  981  985  991  996
 1006 1014 1023 1025 1040 1042 1046 1051 

,count,image_name
0,31,seq_000963.jpg
1,37,seq_000373.jpg
2,33,seq_001265.jpg
3,43,seq_000985.jpg
4,35,seq_000131.jpg
...,...,...
1995,36,seq_001110.jpg
1996,24,seq_001416.jpg
1997,34,seq_001777.jpg
1998,30,seq_001445.jpg


,count,image_name
0,31,seq_000963.jpg
1,37,seq_000373.jpg
5,36,seq_001570.jpg
6,23,seq_001530.jpg
8,43,seq_001618.jpg
...,...,...
1992,25,seq_000502.jpg
1993,29,seq_001778.jpg
1994,30,seq_001747.jpg
1997,34,seq_001777.jpg


Train size: 1600


,count,image_name
2,33,seq_001265.jpg
3,43,seq_000985.jpg
4,35,seq_000131.jpg
7,35,seq_000150.jpg
37,32,seq_001694.jpg
...,...,...
1982,24,seq_000636.jpg
1983,23,seq_000871.jpg
1995,36,seq_001110.jpg
1996,24,seq_001416.jpg


Test size: 400


In [20]:
(train_generator, validation_generator) = training_utils.load_generators(i_train, i_test, dataset_path, batch_size,
                                                                             img_size)

,count,image_name
0,31,seq_000963.jpg
1,37,seq_000373.jpg
5,36,seq_001570.jpg
6,23,seq_001530.jpg
8,43,seq_001618.jpg
...,...,...
1992,25,seq_000502.jpg
1993,29,seq_001778.jpg
1994,30,seq_001747.jpg
1997,34,seq_001777.jpg


image_name
count
Found 1600 validated image filenames.
Found 400 validated image filenames.


In [29]:
conv_base = base_vgg_16_layers(keras.layers.Input(shape=img_size + (img_channels,)))
conv_base.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 240, 320, 3)]     0         
                                                                 
 tf.math.truediv_1 (TFOpLam  (None, 240, 320, 3)       0         
 bda)                                                            
                                                                 
 tf.math.subtract_1 (TFOpLa  (None, 240, 320, 3)       0         
 mbda)                                                           
                                                                 
 vgg16 (Functional)          (None, 7, 10, 512)        14714688  
                                                                 
Total params: 14714688 (56.13 MB)
Trainable params: 14714688 (56.13 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [30]:
feature_train = conv_base.predict(train_generator, 2000, verbose=1)
feature_val = conv_base.predict(validation_generator, 2000, verbose=1)


2024-01-02 13:11:15.747535: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


4/4 [==============================] - 5s 2s/step


In [63]:
feature_train = np.array(feature_train)
feature_val = np.array(feature_val)

In [64]:
print(feature_train.shape)
display(feature_train[0][0])
print(i_train['count'].values)

(1600, 7, 10, 512)


array([[0.        , 0.        , 0.        , ..., 0.        , 0.75454855,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.6557594 ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.6737831 ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.6592682 ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.7134708 ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.72216547,
        0.        ]], dtype=float32)

[31 37 36 ... 30 34 29]


In [65]:
i_model = output_vgg_16_layers(keras.Input(shape=feature_train.shape[1:]))

i_model.compile(loss="mse", optimizer="adam", metrics=["mae"])

In [66]:
dense_history = i_model.fit(feature_train, i_train['count'].values,
            epochs=epochs,
            callbacks=callbacks_list,
            validation_data=(feature_val, i_test['count'].values),
            )

Epoch 1/100
50/50 [==============================] - 1s 10ms/step - loss: 142.7598 - mae: 8.9693 - val_loss: 48.2517 - val_mae: 5.4635
Epoch 2/100
50/50 [==============================] - 0s 7ms/step - loss: 49.5966 - mae: 5.6072 - val_loss: 47.6679 - val_mae: 5.4502
Epoch 3/100
50/50 [==============================] - 0s 7ms/step - loss: 48.9357 - mae: 5.5707 - val_loss: 48.3315 - val_mae: 5.4671
Epoch 4/100
50/50 [==============================] - 0s 6ms/step - loss: 48.9126 - mae: 5.5975 - val_loss: 47.7478 - val_mae: 5.4458
Epoch 5/100
50/50 [==============================] - 0s 6ms/step - loss: 49.4443 - mae: 5.6090 - val_loss: 47.6410 - val_mae: 5.4887
Epoch 6/100
50/50 [==============================] - 0s 6ms/step - loss: 49.2421 - mae: 5.6107 - val_loss: 48.4710 - val_mae: 5.4730
Epoch 7/100
50/50 [==============================] - 0s 7ms/step - loss: 49.2677 - mae: 5.6124 - val_loss: 48.5692 - val_mae: 5.6021
Epoch 8/100
50/50 [==============================] - 0s 6ms/step - 

In [80]:

merged_model = keras.Model(inputs=conv_base.input, outputs=i_model(conv_base.output))
activate_fine_tuning_on_vgg_16(merged_model)
merged_model.summary()

Freezing layer input_7
Freezing layer block1_conv1
Freezing layer block1_conv2
Freezing layer block1_pool
Freezing layer block2_conv1
Freezing layer block2_conv2
Freezing layer block2_pool
Freezing layer block3_conv1
Freezing layer block3_conv2
Freezing layer block3_conv3
Freezing layer block3_pool
Freezing layer block4_conv1
Freezing layer block4_conv2
Freezing layer block4_conv3
Freezing layer block4_pool
Unfreezing layer block5_conv1
Unfreezing layer block5_conv2
Unfreezing layer block5_conv3
Unfreezing layer block5_pool
Model: "model_15"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 240, 320, 3)]     0         
                                                                 
 tf.math.truediv_1 (TFOpLam  (None, 240, 320, 3)       0         
 bda)                                                            
                                                        

In [81]:
merged_model.compile(loss="mse", optimizer="adam", metrics=["mae"])

In [82]:
merged_model.summary()

Model: "model_15"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 240, 320, 3)]     0         
                                                                 
 tf.math.truediv_1 (TFOpLam  (None, 240, 320, 3)       0         
 bda)                                                            
                                                                 
 tf.math.subtract_1 (TFOpLa  (None, 240, 320, 3)       0         
 mbda)                                                           
                                                                 
 vgg16 (Functional)          (None, 7, 10, 512)        14714688  
                                                                 
 model_12 (Functional)       (None, 1)                 35841     
                                                                 
Total params: 14750529 (56.27 MB)
Trainable params: 711526

In [ ]:
conv_history = merged_model.fit(train_generator,
            epochs=epochs,
            callbacks=callbacks_list,
            validation_data=validation_generator,
            )

Epoch 1/100
13/13 [==============================] - 31s 2s/step - loss: 239.2250 - mae: 12.2168 - val_loss: 56.2990 - val_mae: 6.1578
Epoch 2/100
13/13 [==============================] - 29s 2s/step - loss: 54.9766 - mae: 5.9651 - val_loss: 48.4148 - val_mae: 5.4688
Epoch 3/100
13/13 [==============================] - ETA: 0s - loss: 54.0200 - mae: 5.8453INFO:tensorflow:Assets written to: VGG16_(256).tf/assets


INFO:tensorflow:Assets written to: VGG16_(256).tf/assets


13/13 [==============================] - 30s 2s/step - loss: 54.0200 - mae: 5.8453 - val_loss: 47.4642 - val_mae: 5.4760
Epoch 4/100
13/13 [==============================] - 30s 2s/step - loss: 52.4187 - mae: 5.8102 - val_loss: 49.8784 - val_mae: 5.5144
Epoch 5/100
13/13 [==============================] - 30s 2s/step - loss: 51.3159 - mae: 5.7544 - val_loss: 47.4689 - val_mae: 5.4519
Epoch 6/100
13/13 [==============================] - 29s 2s/step - loss: 50.4804 - mae: 5.6598 - val_loss: 47.5284 - val_mae: 5.4467
Epoch 7/100
13/13 [==============================] - 29s 2s/step - loss: 50.1225 - mae: 5.6721 - val_loss: 47.4979 - val_mae: 5.4794
Epoch 8/100
13/13 [==============================] - ETA: 0s - loss: 49.5820 - mae: 5.6346INFO:tensorflow:Assets written to: VGG16_(256).tf/assets


INFO:tensorflow:Assets written to: VGG16_(256).tf/assets


13/13 [==============================] - 31s 2s/step - loss: 49.5820 - mae: 5.6346 - val_loss: 47.4446 - val_mae: 5.4744
Epoch 9/100
13/13 [==============================] - 29s 2s/step - loss: 49.3880 - mae: 5.6167 - val_loss: 48.0670 - val_mae: 5.4534
Epoch 10/100
13/13 [==============================] - 29s 2s/step - loss: 48.8608 - mae: 5.5440 - val_loss: 48.0357 - val_mae: 5.4518
Epoch 11/100
13/13 [==============================] - 29s 2s/step - loss: 52.9770 - mae: 5.8214 - val_loss: 58.6828 - val_mae: 5.9530
Epoch 12/100
13/13 [==============================] - 30s 2s/step - loss: 51.7582 - mae: 5.7425 - val_loss: 47.4875 - val_mae: 5.4487
Epoch 13/100
13/13 [==============================] - 29s 2s/step - loss: 52.4475 - mae: 5.7717 - val_loss: 48.0432 - val_mae: 5.4522
Epoch 14/100
13/13 [==============================] - 29s 2s/step - loss: 49.3710 - mae: 5.6070 - val_loss: 47.5016 - val_mae: 5.4797
Epoch 15/100
13/13 [==============================] - 30s 2s/step - loss: 49

INFO:tensorflow:Assets written to: VGG16_(256).tf/assets


13/13 [==============================] - 30s 2s/step - loss: 49.3274 - mae: 5.6137 - val_loss: 47.3917 - val_mae: 5.4624
Epoch 21/100
13/13 [==============================] - 29s 2s/step - loss: 49.1467 - mae: 5.6093 - val_loss: 48.1539 - val_mae: 5.4574
Epoch 22/100
13/13 [==============================] - 29s 2s/step - loss: 49.1820 - mae: 5.6079 - val_loss: 47.4707 - val_mae: 5.4775
Epoch 23/100
13/13 [==============================] - 29s 2s/step - loss: 49.2955 - mae: 5.6167 - val_loss: 48.7785 - val_mae: 5.4818
Epoch 24/100
13/13 [==============================] - 30s 2s/step - loss: 49.4876 - mae: 5.6126 - val_loss: 47.4031 - val_mae: 5.4561
Epoch 25/100
13/13 [==============================] - 29s 2s/step - loss: 49.6850 - mae: 5.6255 - val_loss: 49.6227 - val_mae: 5.5070
Epoch 26/100
13/13 [==============================] - 29s 2s/step - loss: 49.8279 - mae: 5.6337 - val_loss: 47.4818 - val_mae: 5.4467
Epoch 27/100
13/13 [==============================] - 28s 2s/step - loss: 5

# Represent history

In [ ]:
print(history)
mea_results = [np.min(x.history['mae']) for x in history]
val_mea_results = [np.min(x.history['val_mae']) for x in history]
loss_results = [np.min(x.history['loss']) for x in history]
val_loss_results = [np.min(x.history['val_loss']) for x in history]
print(mea_results)

In [ ]:
print(f'Mean mae: {np.mean(mea_results)}')
print(f'Mean val_mae: {np.mean(val_mea_results)}')
print(f'Mean loss: {np.mean(loss_results)}')
print(f'Mean val_loss: {np.mean(val_loss_results)}')

In [ ]:
print(history[0].history.keys())
history_single = history[0]

In [ ]:
# history_single = history

In [ ]:
plt.plot(history_single.history['mae'][display_epochs[0]:display_epochs[1]])
plt.plot(history_single.history['val_mae'][display_epochs[0]:display_epochs[1]])
plt.title('model mean squared')
plt.ylabel('mean squared')
plt.xlabel('Epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history_single.history['loss'][display_epochs[0]:display_epochs[1]])
plt.plot(history_single.history['val_loss'][display_epochs[0]:display_epochs[1]])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

# Bibliografia

- Deep Learning with Python, Second Edition. François Chollet
- http://www.crowd-counting.com/
- https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html

Dataset citation:
- From Semi-Supervised to Transfer Counting of Crowds
C. C. Loy, S. Gong, and T. Xiang
in Proceedings of IEEE International Conference on Computer Vision, pp. 2256-2263, 2013 (ICCV)
- Cumulative Attribute Space for Age and Crowd Density Estimation
K. Chen, S. Gong, T. Xiang, and C. C. Loy
in Proceedings of IEEE Conference on Computer Vision and Pattern Recognition, pp. 2467-2474, 2013 (CVPR, Oral)
- Crowd Counting and Profiling: Methodology and Evaluation
C. C. Loy, K. Chen, S. Gong, T. Xiang
in S. Ali, K. Nishino, D. Manocha, and M. Shah (Eds.), Modeling, Simulation and Visual Analysis of Crowds, Springer, vol. 11, pp. 347-382, 2013
- Feature Mining for Localised Crowd Counting
K. Chen, C. C. Loy, S. Gong, and T. Xiang
British Machine Vision Conference, 2012 (BMVC)


@inproceedings{sindagi2019pushing,
title={Pushing the frontiers of unconstrained crowd counting: New dataset and benchmark method},
author={Sindagi, Vishwanath A and Yasarla, Rajeev and Patel, Vishal M},
booktitle={Proceedings of the IEEE International Conference on Computer Vision},
pages={1221--1231},
year={2019}
}
@article{sindagi2020jhu-crowd++,
title={JHU-CROWD++: Large-Scale Crowd Counting Dataset and A Benchmark Method},
author={Sindagi, Vishwanath A and Yasarla, Rajeev and Patel, Vishal M},
journal={Technical Report},
year={2020}
}